In [21]:
import pandas as pd, numpy as np
import re

# Non historical Data

## 1. Get Country Data
Source: https://www.kaggle.com/fernandol/countries-of-the-world

In [22]:
countries = pd.read_csv("Sources//countries-of-the-world.csv", na_values=("NaN"), decimal=",", skipinitialspace=True)

Replace spaces with Regex and rewrite columns

In [23]:
countries.Country = countries.Country.replace(to_replace={"(\W\s?\W?)\Z" : ""}, regex=True)

In [24]:
countries_drop = countries.drop(columns=["Coastline (coast/area ratio)", 
                                         "Net migration", 
                                         "Infant mortality (per 1000 births)", 
                                         "Phones (per 1000)", 
                                         "Crops (%)", 
                                         "Other (%)", 
                                         "Agriculture", 
                                         "Industry", 
                                         "Service",
                                         "Climate"])

In [25]:
countries_drop = countries_drop.replace(to_replace={'ASIA (EX. NEAR EAST)         ': "ASIA"
                                        ,'BALTICS                            ' : "BALTICS"
                                        ,'C.W. OF IND. STATES ' : "C.W. OF IND. STATES"
                                        ,'EASTERN EUROPE                     ' : "EASTERN EUROPE"
                                        ,'LATIN AMER. & CARIB    ' : "LATIN AMER. & CARIB"
                                        ,'NEAR EAST                          ' : "NEAR EAST"
                                        ,'NORTHERN AFRICA                    ' : "NORTHERN AFRICA"
                                        ,'NORTHERN AMERICA                   ' :  "NORTHERN AMERICA"
                                        ,'OCEANIA                            ' : "OCEANIA"
                                        ,'SUB-SAHARAN AFRICA                 ' : "SUB-SAHARAN AFRICA"
                                        ,'WESTERN EUROPE                     ' :  "WESTERN EUROPE"})

In [26]:
Data_Full = countries_drop.copy()

# Historical Data

## 1. Get Temperature
Source: http://apps.ipcc-data.org/observ/clim/cru_ts2_1.html

In [27]:
temperature_1961_1990_country = pd.read_csv("Sources//Temp-1961-1990-Country.csv", 
                                            sep=";",
                                            decimal=",")

***Merge***

In [28]:
Data_Full = Data_Full.merge(temperature_1961_1990_country, how="left", on="Country")

## 2. Get Precipitation
Source: https://databank.worldbank.org/source/world-development-indicators#

In [29]:
precipitation_1984_2019 = pd.read_csv("Sources//Precipitation_1984-2019.csv", 
                                      sep=",", 
                                      na_values="..")

precipitation_1984_2019 = precipitation_1984_2019.drop(columns=["1984 [YR1984]", 
                                                                "1988 [YR1988]", 
                                                                "1998 [YR1998]", 
                                                                "2006 [YR2006]", 
                                                                "2010 [YR2010]", 
                                                                "2018 [YR2018]"])

precipitation_1984_2019 = (precipitation_1984_2019.drop(columns=["Series Name", "Series Code"]).
                           rename(columns={'1992 [YR1992]' : "precipitation mm/y 1992", 
                                           '2002 [YR2002]' : "precipitation mm/y 2002", 
                                           '2014 [YR2014]' : "precipitation mm/y 2014",
                                           'Country Name' : "Country"}))

***Merge***

In [30]:
Data_Full = Data_Full.merge(precipitation_1984_2019, how="left", on="Country")

## 3. Get Water
Source: https://databank.worldbank.org/source/world-development-indicators#

### Consumption

In [31]:
renewable_freshwater = pd.read_csv("Sources//renewable_freshwater.csv",
                         sep=",",
                         na_values="..")


In [32]:
renewable_freshwater = (renewable_freshwater.drop(columns=["1984 [YR1984]",
                                                           "1988 [YR1988]", 
                                                           "1998 [YR1998]", 
                                                           "2006 [YR2006]", 
                                                           "2010 [YR2010]", 
                                                           "2018 [YR2018]",
                                                           "Unnamed: 0",
                                                           "Country Code",
                                                           "Series Name",
                                                           "Series Code"]).
                     rename(columns={'1992 [YR1992]' : "renewable freshwater m3/capita 1992", 
                                     '2002 [YR2002]' : "renewable freshwater m3/capita 2002",
                                     '2014 [YR2014]' : "renewable freshwater m3/capita 2014",
                                     'Country Name' : "Country"}))

***Merge***

In [33]:
Data_Full = Data_Full.merge(renewable_freshwater, how="left", on="Country")

### Withdraw

In [34]:
water_withdraw = pd.read_csv("Sources//water_withdraw.csv",
                             sep=",",
                             na_values="..",
                             )

In [35]:
water_withdraw = (water_withdraw.drop(columns=['1984 [YR1984]', '1988 [YR1988]', '1992 [YR1992]',
       '1998 [YR1998]', '2006 [YR2006]', '2010 [YR2010]', 'Series Code',
       '2014 [YR2014]', '2018 [YR2018]', "Unnamed: 0", "Country Code", "Series Name"]).
                 rename(columns={"2002 [YR2002]" : "annual water withdraw % 2002", "Country Name" : "Country"}))

***Merge***

In [36]:
Data_Full = Data_Full.merge(water_withdraw, how="left", on="Country")

In [37]:
Data_Full

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),GDP ($ per capita),Literacy (%),Arable (%),Birthrate,Deathrate,Temp C° 1961-1990,Country Code,precipitation mm/y 1992,precipitation mm/y 2002,precipitation mm/y 2014,renewable freshwater m3/capita 1992,renewable freshwater m3/capita 2002,renewable freshwater m3/capita 2014,annual water withdraw % 2002
0,Afghanistan,ASIA,31056997,647500,48.0,700.0,36.0,12.13,46.60,20.34,12.60,AFG,327.0,327.0,327.0,3254.968781,2086.212107,1412.912141,43.011665
1,Albania,EASTERN EUROPE,3581655,28748,124.6,4500.0,86.5,21.09,15.11,5.22,11.40,ALB,1485.0,1485.0,1485.0,8284.470867,8816.752485,9310.845162,6.832714
2,Algeria,NORTHERN AFRICA,32930091,2381740,13.8,6000.0,70.0,3.22,17.14,4.61,22.50,DZA,89.0,89.0,89.0,416.229995,353.161560,289.027090,50.720000
3,American Samoa,OCEANIA,57794,199,290.4,8000.0,97.0,10.00,22.46,3.27,NaN,ASM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,152.1,19000.0,100.0,2.22,8.71,6.25,7.60,AND,NaN,NaN,NaN,5359.326179,4505.417636,3984.194514,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,West Bank,NEAR EAST,2460492,5860,419.9,800.0,NaN,16.90,31.67,3.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,Western Sahara,NORTHERN AFRICA,273008,266000,1.0,NaN,NaN,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,Yemen,NEAR EAST,21456188,527970,40.6,800.0,50.2,2.78,42.89,8.30,23.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,Zambia,SUB-SAHARAN AFRICA,11502010,752614,15.3,800.0,80.6,7.08,41.00,19.93,21.40,ZMB,1020.0,1020.0,1020.0,9489.611538,7309.716326,5207.875737,1.960100


# Save to .csv

In [38]:
Data_Full.to_csv("Data//Full_Data.csv")